<h3>Setup</h3>

In [1]:
# Imports
import kagglehub
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
# Load Kaggle dataset
def load_kaggle_csv(dataset, filename):
    path = Path(kagglehub.dataset_download(dataset))
    return pd.read_csv(path / filename)

df = load_kaggle_csv(
    "chitwanmanchanda/fraudulent-transactions-data",
    "Fraud.csv"
)

# Sort by time
df = df.sort_values('step').reset_index(drop=True)

<h3>Feature Engineering</h3>

In [3]:
# Calculate change in balance for both accounts
df['orig_delta'] = df['newbalanceOrig'] - df['oldbalanceOrg']
df['dest_delta'] = df['newbalanceDest'] - df['oldbalanceDest']

# Calculate change in balance for origin
mask_orig_customer = ~df['nameOrig'].str.startswith('M')
df['orig_delta'] = 0  # default 0 for merchants
df.loc[mask_orig_customer, 'orig_delta'] = (
    df.loc[mask_orig_customer, 'newbalanceOrig'] - 
    df.loc[mask_orig_customer, 'oldbalanceOrg']
)

# Calculate change in balance for destination
mask_dest_customer = ~df['nameDest'].str.startswith('M')
df['dest_delta'] = 0  # default 0 for merchants
df.loc[mask_dest_customer, 'dest_delta'] = (
    df.loc[mask_dest_customer, 'newbalanceDest'] - 
    df.loc[mask_dest_customer, 'oldbalanceDest']
)

# Initialize previous fraud flag
df['orig_prev_fraud'] = 0
df['dest_prev_fraud'] = 0

# Origin previous fraud
df['orig_prev_fraud'] = df.groupby('nameOrig')['isFraud'].cumsum().shift(fill_value=0)
df['orig_prev_fraud'] = (df['orig_prev_fraud'] > 0).astype(int)

# Destination previous fraud
df['dest_prev_fraud'] = df.groupby('nameDest')['isFraud'].cumsum().shift(fill_value=0)
df['dest_prev_fraud'] = (df['dest_prev_fraud'] > 0).astype(int)

# Drop balance figures and answer column
df = df.drop(columns=[
    'oldbalanceOrg', 'newbalanceOrig',
    'oldbalanceDest', 'newbalanceDest',
    'type','isFlaggedFraud'
])

# Apply one hot encording for transaction type
pd.get_dummies(df, columns=['type'], drop_first=True)

KeyError: "None of [Index(['type'], dtype='object')] are in the [columns]"

In [ ]:
df.head()

<h3>Machine Learning</h3>

In [ ]:
X = df.drop('isFraud', axis=1)
y = df['isFraud']

split_index = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

print(f"X_train shape {X_train.shape}")
print(f"X_test shape {X_test.shape}")

In [ ]:
X_train

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
linkcode
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))